# Student Performance Interactive Dashboard - Plotly & Dash Analysis

## Data Overview
This interactive analysis is based on learning performance data from 1000 students, including the following features:
- Gender
- Race/ethnicity
- Parental level of education
- Lunch type
- Test preparation course completion status
- Math score
- Reading score
- Writing score

**Features:** Interactive visualizations with hover data, filtering, and real-time updates using Plotly and Dash.
#Test

In [1]:
!pip3 install nbformat

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set default theme for Plotly
import plotly.io as pio
pio.templates.default = "plotly_white"

In [3]:
%pip install nbformat>=4.2.0

import nbformat  # Ensure nbformat is imported

# Load data
df = pd.read_csv('/Users/ConstiX/Library/Mobile Documents/com~apple~CloudDocs/Career and Academics/Master/Python Bootcamp/StudentsPerformance.csv')
print(f"Dataset size: {df.shape[0]} rows, {df.shape[1]} columns")
print("\nFirst 5 rows of data:")

# Display interactive table
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df[col] for col in df.columns],
               fill_color='lavender',
               align='left'))
])
fig.update_layout(title="Interactive Dataset Preview")
fig.show()

df.head()

zsh:1: 4.2.0 not found
Note: you may need to restart the kernel to use updated packages.
Dataset size: 1000 rows, 8 columns

First 5 rows of data:
Note: you may need to restart the kernel to use updated packages.
Dataset size: 1000 rows, 8 columns

First 5 rows of data:


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
# Basic information
print("Basic data information:")
df.info()
print("\nMissing values statistics:")
print(df.isna().sum())
print("\nDescriptive statistics for numerical variables:")

# Interactive descriptive statistics
stats_df = df.describe()
fig = go.Figure(data=[go.Table(
    header=dict(values=['Statistic'] + list(stats_df.columns),
                fill_color='lightblue',
                align='left'),
    cells=dict(values=[list(stats_df.index)] + [stats_df[col].round(2) for col in stats_df.columns],
               fill_color='lightcyan',
               align='left'))
])
fig.update_layout(title="Interactive Descriptive Statistics")
fig.show()

df.describe()

Basic data information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB

Missing values statistics:
gender                         0
race/ethnicity                 0
parental level of education    0
lunch                          0
test preparation course        0
math score                     0
reading score           

,math score,reading score,writing score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


## 1. Data Cleaning and Preprocessing

In [5]:
# Rename columns for easier analysis
df.columns = ['gender', 'race_ethnicity', 'parental_education', 'lunch', 'test_prep', 'math_score', 'reading_score', 'writing_score']

# Calculate average score
df['average_score'] = (df['math_score'] + df['reading_score'] + df['writing_score']) / 3

# Create grade levels
def score_grade(score):
    if score >= 90:
        return 'A'
    elif score >= 80:
        return 'B'
    elif score >= 70:
        return 'C'
    elif score >= 60:
        return 'D'
    else:
        return 'F'

df['grade'] = df['average_score'].apply(score_grade)

print("Data after preprocessing:")

# Interactive preview of processed data
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='lightgreen',
                align='left'),
    cells=dict(values=[df[col].head() for col in df.columns],
               fill_color='lightgray',
               align='left'))
])
fig.update_layout(title="Processed Data Preview")
fig.show()

df.head()

Data after preprocessing:


,gender,race_ethnicity,parental_education,lunch,test_prep,math_score,reading_score,writing_score,average_score,grade
0,female,group B,bachelor's degree,standard,none,72,72,74,72.666667,C
1,female,group C,some college,standard,completed,69,90,88,82.333333,B
2,female,group B,master's degree,standard,none,90,95,93,92.666667,A
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.333333,F
4,male,group C,some college,standard,none,76,78,75,76.333333,C


## 2. Interactive Exploratory Data Analysis

In [6]:
# Interactive distribution of categorical variables
categorical_cols = ['gender', 'race_ethnicity', 'parental_education', 'lunch', 'test_prep', 'grade']

# Create interactive pie charts with dropdown selector
fig = make_subplots(rows=2, cols=3, 
                    specs=[[{'type':'domain'}]*3, [{'type':'domain'}]*3],
                    subplot_titles=[col.replace('_', ' ').title() for col in categorical_cols])

colors_palette = px.colors.qualitative.Set3

for i, col in enumerate(categorical_cols):
    row = (i // 3) + 1
    col_idx = (i % 3) + 1
    
    counts = df[col].value_counts()
    
    fig.add_trace(go.Pie(
        labels=counts.index,
        values=counts.values,
        name=col.replace('_', ' ').title(),
        hovertemplate="<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>",
        textinfo='label+percent',
        textposition='auto',
        marker_colors=colors_palette[:len(counts)]
    ), row=row, col=col_idx)

fig.update_layout(
    title_text="Interactive Categorical Variables Distribution",
    title_x=0.5,
    height=800,
    showlegend=False
)

fig.show()

# Updated dashboard logic for selection, multi-factor analysis, gender color, hover, and box plot repetition
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import pandas as pd

# Assume df is your DataFrame and 'Gender' is a column
app = Dash(__name__)
app.layout = html.Div([
    dcc.Dropdown(
        id='feature-dropdown',
        options=[{'label': col, 'value': col} for col in df.columns if col != 'Gender'],
        value=df.columns[0],
        clearable=False, # Only one selection at a time
        style={'width': '50%'},
    ),
    dcc.Graph(id='box-plot'),
    dcc.Graph(id='multi-factor-plot'),
])

@app.callback(
    Output('box-plot', 'figure'),
    Output('multi-factor-plot', 'figure'),
    Input('feature-dropdown', 'value')
)
def update_dashboard(selected_feature):
    # Box plot: differentiate genders by color, do not repeat
    box_fig = px.box(
        df, x='Gender', y=selected_feature, color='Gender',
        color_discrete_map={'Male': 'blue', 'Female': 'pink'},
        points='all',
        hover_data=df.columns,
        title=f'Box Plot of {selected_feature} by Gender'
    )
    box_fig.update_traces(hovertemplate=
        '<b>Gender:</b> %{x}<br>' +
        f'<b>{selected_feature}:</b> %{{y}}<br>' +
        '<b>Other:</b> %{customdata}',
        marker=dict(size=8)
    )
    box_fig.update_layout(hoverlabel_font_size=14)

    # Multi-factor analysis: example with scatter plot (fix/clarify)
    # You can replace with your actual analysis logic
    multi_factor_fig = px.scatter(
        df, x=selected_feature, y='score', color='Gender',
        color_discrete_map={'Male': 'blue', 'Female': 'pink'},
        hover_data=df.columns,
        title=f'Multi-Factor Analysis: {selected_feature} vs Score by Gender'
    )
    multi_factor_fig.update_traces(marker=dict(size=10),
        hovertemplate=
        f'<b>{selected_feature}:</b> %{{x}}<br>' +
        '<b>Score:</b> %{y}<br>' +
        '<b>Gender:</b> %{marker.color}',
    )
    multi_factor_fig.update_layout(hoverlabel_font_size=14)

    return box_fig, multi_factor_fig

# To run the app:
# if __name__ == '__main__':
#     app.run_server(debug=True)

In [7]:
# Interactive score distribution with proper 2x2 layout
score_cols = ['math_score', 'reading_score', 'writing_score', 'average_score']
colors = px.colors.qualitative.Set2

fig = make_subplots(rows=2, cols=2,
                    subplot_titles=[col.replace('_', ' ').title() + ' Distribution' for col in score_cols],
                    vertical_spacing=0.12,
                    horizontal_spacing=0.08)

for i, col in enumerate(score_cols):
    row = (i // 2) + 1
    col_idx = (i % 2) + 1
    
    # Add histogram
    fig.add_trace(go.Histogram(
        x=df[col],
        name=col.replace('_', ' ').title(),
        nbinsx=20,
        marker_color=colors[i],
        opacity=0.7,
        showlegend=False,
        hovertemplate="<b>Score Range:</b> %{x}<br><b>Count:</b> %{y}<br><b>Percentage:</b> %{y}<extra></extra>"
    ), row=row, col=col_idx)
    
    # Add mean line using shapes (works better with subplots)
    mean_val = df[col].mean()
    std_val = df[col].std()
    
    # Add vertical line for mean
    fig.add_shape(
        type="line",
        x0=mean_val, x1=mean_val,
        y0=0, y1=1,
        yref=f"y{i+1 if i > 0 else ''} domain",
        line=dict(color="red", width=2, dash="dash"),
        row=row, col=col_idx
    )
    
    # Add mean annotation
    fig.add_annotation(
        x=mean_val,
        y=0.9,
        yref=f"y{i+1 if i > 0 else ''} domain",
        text=f"Mean: {mean_val:.1f}",
        showarrow=False,
        font=dict(size=10, color="red"),
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="red",
        borderwidth=1,
        row=row, col=col_idx
    )
    
    # Add standard deviation annotation
    fig.add_annotation(
        x=mean_val,
        y=0.8,
        yref=f"y{i+1 if i > 0 else ''} domain",
        text=f"Std: {std_val:.1f}",
        showarrow=False,
        font=dict(size=9, color="blue"),
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="blue",
        borderwidth=1,
        row=row, col=col_idx
    )

# Update layout for better appearance
fig.update_layout(
    title_text="Interactive Score Distributions with Statistics",
    title_x=0.5,
    height=700,
    showlegend=False
)

# Update all x-axes with better labels
fig.update_xaxes(title_text="Score", showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(title_text="Frequency", showgrid=True, gridwidth=1, gridcolor='lightgray')

# Add interactive features (zoom and pan work better than range sliders for subplots)
fig.update_layout(
    dragmode='zoom',
    hovermode='closest'
)

fig.show()

## 3. Interactive Key Findings Analysis

In [8]:
# Interactive gender performance analysis
df_melted = df.melt(id_vars=['gender'], value_vars=['math_score', 'reading_score', 'writing_score'],
                   var_name='subject', value_name='score')

# Create interactive box plots
fig1 = px.box(df_melted, x='subject', y='score', color='gender',
              title="Interactive Score Distribution by Gender Across Subjects",
              hover_data=['gender'])
fig1.update_traces(quartilemethod="exclusive")
fig1.update_layout(height=500)

# Statistical testing and display
test_results = []
for subject in ['math_score', 'reading_score', 'writing_score']:
    male_scores = df[df['gender'] == 'male'][subject]
    female_scores = df[df['gender'] == 'female'][subject]
    t_stat, p_value = stats.ttest_ind(male_scores, female_scores)
    test_results.append({
        'Subject': subject.replace('_', ' ').title(),
        'T-Statistic': round(t_stat, 3),
        'P-Value': round(p_value, 3),
        'Significant': 'Yes' if p_value < 0.05 else 'No'
    })

# Add statistical results as annotation
results_text = "<br>".join([f"{r['Subject']}: t={r['T-Statistic']}, p={r['P-Value']} ({'*' if r['Significant']=='Yes' else 'ns'})" 
                          for r in test_results])
fig1.add_annotation(
    text=f"<b>Statistical Tests (α=0.05):</b><br>{results_text}",
    xref="paper", yref="paper",
    x=1.02, y=0.98,
    showarrow=False,
    align="left",
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="gray",
    borderwidth=1
)

fig1.show()

# Interactive average scores comparison
gender_scores = df.groupby('gender')[['math_score', 'reading_score', 'writing_score', 'average_score']].mean()
fig2 = px.bar(gender_scores.reset_index(), x='gender', 
              y=['math_score', 'reading_score', 'writing_score', 'average_score'],
              title="Interactive Average Scores by Gender",
              barmode='group',
              hover_data={'variable': True, 'value': ':.2f'})
fig2.update_layout(height=400)
fig2.show()

print("Gender Analysis Summary:")
for result in test_results:
    print(f"  {result['Subject']}: t-statistic={result['T-Statistic']}, p-value={result['P-Value']}")

Gender Analysis Summary:
  Math Score: t-statistic=5.383, p-value=0.0
  Reading Score: t-statistic=-7.959, p-value=0.0
  Writing Score: t-statistic=-9.98, p-value=0.0


In [9]:
# Interactive parental education impact analysis
education_order = ['some high school', 'high school', 'some college', "associate's degree", "bachelor's degree", "master's degree"]
df['parental_education'] = pd.Categorical(df['parental_education'], categories=education_order, ordered=True)

# Interactive box plot with trend line
fig = px.box(df, x='parental_education', y='average_score',
             title="Interactive Impact of Parental Education Level on Student Average Scores",
             hover_data=['parental_education', 'average_score'])

# Add trend line
edu_means = df.groupby('parental_education')['average_score'].mean()
fig.add_trace(go.Scatter(
    x=education_order,
    y=edu_means.values,
    mode='lines+markers',
    name='Trend Line',
    line=dict(color='red', width=3),
    marker=dict(size=8),
    hovertemplate="<b>Education Level:</b> %{x}<br><b>Average Score:</b> %{y:.2f}<extra></extra>"
))

fig.update_xaxes(title="Parental Education Level")
fig.update_yaxes(title="Average Score")
fig.update_layout(height=600, xaxis_tickangle=45)

# Calculate and display correlation
edu_scores = df.groupby('parental_education')['average_score'].mean().reset_index()
edu_scores['edu_numeric'] = range(len(edu_scores))
correlation = edu_scores['edu_numeric'].corr(edu_scores['average_score'])

fig.add_annotation(
    text=f"<b>Correlation with Education Level:</b> {correlation:.3f}",
    xref="paper", yref="paper",
    x=0.02, y=0.98,
    showarrow=False,
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="blue",
    borderwidth=1
)

fig.show()

print(f"Correlation between parental education level and student performance: {correlation:.3f}")

Correlation between parental education level and student performance: 0.939


In [10]:
# Interactive lunch type (economic status) analysis
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=["Box Plot Analysis", "Violin Plot with Distribution"])

# Box plot
for i, lunch_type in enumerate(df['lunch'].unique()):
    data = df[df['lunch'] == lunch_type]['average_score']
    fig.add_trace(go.Box(
        y=data,
        name=lunch_type,
        boxpoints='outliers',
        hovertemplate="<b>Lunch Type:</b> %{x}<br><b>Score:</b> %{y}<br><b>Count:</b> %{text}<extra></extra>",
        text=[lunch_type] * len(data)
    ), row=1, col=1)

# Violin plot
for i, lunch_type in enumerate(df['lunch'].unique()):
    data = df[df['lunch'] == lunch_type]['average_score']
    fig.add_trace(go.Violin(
        y=data,
        name=lunch_type,
        box_visible=True,
        meanline_visible=True,
        hovertemplate="<b>Lunch Type:</b> %{x}<br><b>Score:</b> %{y}<extra></extra>"
    ), row=1, col=2)

fig.update_layout(
    title_text="Interactive Impact of Lunch Type on Average Scores",
    height=500,
    showlegend=False
)

fig.update_yaxes(title="Average Score")
fig.update_xaxes(title="Lunch Type")

# Statistical testing
standard_lunch = df[df['lunch'] == 'standard']['average_score']
reduced_lunch = df[df['lunch'] == 'free/reduced']['average_score']
t_stat, p_value = stats.ttest_ind(standard_lunch, reduced_lunch)

fig.add_annotation(
    text=f"<b>Statistical Test Results:</b><br>t-statistic: {t_stat:.3f}<br>p-value: {p_value:.3f}<br><br><b>Mean Scores:</b><br>Standard: {standard_lunch.mean():.2f}<br>Free/Reduced: {reduced_lunch.mean():.2f}<br>Difference: {standard_lunch.mean() - reduced_lunch.mean():.2f}",
    xref="paper", yref="paper",
    x=1.02, y=0.98,
    showarrow=False,
    align="left",
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="orange",
    borderwidth=1
)

fig.show()

print(f"Lunch type difference test: t-statistic={t_stat:.3f}, p-value={p_value:.3f}")
print(f"Standard lunch students average score: {standard_lunch.mean():.2f}")
print(f"Free/reduced lunch students average score: {reduced_lunch.mean():.2f}")

Lunch type difference test: t-statistic=9.575, p-value=0.000
Standard lunch students average score: 70.84
Free/reduced lunch students average score: 62.20


In [11]:
# Interactive test preparation course impact analysis
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=["Subject Scores by Test Prep Status", "Score Improvement from Test Prep"])

# Average score comparison
prep_scores = df.groupby('test_prep')[['math_score', 'reading_score', 'writing_score']].mean()
prep_data = prep_scores.reset_index().melt(id_vars=['test_prep'], var_name='subject', value_name='score')

for subject in ['math_score', 'reading_score', 'writing_score']:
    subject_data = prep_data[prep_data['subject'] == subject]
    fig.add_trace(go.Bar(
        x=subject_data['test_prep'],
        y=subject_data['score'],
        name=subject.replace('_', ' ').title(),
        hovertemplate="<b>Test Prep:</b> %{x}<br><b>Subject:</b> " + subject.replace('_', ' ').title() + "<br><b>Average Score:</b> %{y:.2f}<extra></extra>"
    ), row=1, col=1)

# Score improvement magnitude
categories = ['Math', 'Reading', 'Writing']
subjects = ['math_score', 'reading_score', 'writing_score']
improvements = []

for subject in subjects:
    comp_score = df[df['test_prep'] == 'completed'][subject].mean()
    none_score = df[df['test_prep'] == 'none'][subject].mean()
    improvements.append(comp_score - none_score)

fig.add_trace(go.Bar(
    x=categories,
    y=improvements,
    name='Score Improvement',
    marker_color=['skyblue', 'lightgreen', 'lightcoral'],
    hovertemplate="<b>Subject:</b> %{x}<br><b>Improvement:</b> %{y:.2f} points<extra></extra>"
), row=1, col=2)

# Add horizontal line at zero
fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5, row=1, col=2)

fig.update_layout(
    title_text="Interactive Test Preparation Course Impact Analysis",
    height=500,
    barmode='group'
)

fig.update_xaxes(title="Test Preparation Status", row=1, col=1)
fig.update_xaxes(title="Subject", row=1, col=2)
fig.update_yaxes(title="Average Score", row=1, col=1)
fig.update_yaxes(title="Score Improvement (Points)", row=1, col=2)

fig.show()

# Calculate overall improvement
completed = df[df['test_prep'] == 'completed']['average_score'].mean()
none = df[df['test_prep'] == 'none']['average_score'].mean()
overall_improvement = completed - none

print(f"Overall average improvement from test preparation: {overall_improvement:.2f} points")
for i, subject in enumerate(['Math', 'Reading', 'Writing']):
    print(f"{subject} score improvement: {improvements[i]:.2f} points")

Overall average improvement from test preparation: 7.63 points
Math score improvement: 5.62 points
Reading score improvement: 7.36 points
Writing score improvement: 9.91 points


In [12]:
# Interactive race/ethnicity performance analysis
race_scores = df.groupby('race_ethnicity')['average_score'].mean().sort_values(ascending=False)

fig = px.bar(race_scores.reset_index(), x='race_ethnicity', y='average_score',
             title="Interactive Average Scores by Race/Ethnicity",
             color='average_score',
             color_continuous_scale='Viridis',
             hover_data={'average_score': ':.2f'})

# Add value labels on bars
for i, (race, score) in enumerate(race_scores.items()):
    fig.add_annotation(
        x=race,
        y=score + 0.5,
        text=f'{score:.1f}',
        showarrow=False,
        font=dict(size=12, color='black')
    )

# Add ranking information
fig.add_annotation(
    text="<b>Performance Ranking:</b><br>" + "<br>".join([f"{i+1}. {race}: {score:.2f}" for i, (race, score) in enumerate(race_scores.items())]),
    xref="paper", yref="paper",
    x=1.02, y=0.98,
    showarrow=False,
    align="left",
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="purple",
    borderwidth=1
)

fig.update_xaxes(title="Race/Ethnicity", tickangle=45)
fig.update_yaxes(title="Average Score")
fig.update_layout(height=600, coloraxis_showscale=False)

fig.show()

print("Average score ranking by ethnicity:")
for i, (race, score) in enumerate(race_scores.items(), 1):
    print(f"{i}. {race}: {score:.2f}")

Average score ranking by ethnicity:
1. group E: 72.75
2. group D: 69.18
3. group C: 67.13
4. group B: 65.47
5. group A: 62.99


## 4. Interactive Correlation Analysis

In [13]:
# Interactive correlation heatmap
score_correlation = df[['math_score', 'reading_score', 'writing_score']].corr()

# Create interactive heatmap
fig = px.imshow(score_correlation,
                text_auto=True,
                aspect="auto",
                color_continuous_scale='RdBu_r',
                title="Interactive Subject Score Correlation Heatmap")

# Customize the heatmap
fig.update_traces(
    hovertemplate="<b>%{x} vs %{y}</b><br>Correlation: %{z:.3f}<extra></extra>",
    texttemplate="%{z:.3f}",
    textfont={"size": 14}
)

fig.update_layout(
    width=600, 
    height=600,
    xaxis_title="Subjects",
    yaxis_title="Subjects"
)

# Add correlation strength interpretation
fig.add_annotation(
    text="<b>Correlation Strength:</b><br>0.8-1.0: Very Strong<br>0.6-0.8: Strong<br>0.4-0.6: Moderate<br>0.2-0.4: Weak<br>0.0-0.2: Very Weak",
    xref="paper", yref="paper",
    x=1.15, y=0.98,
    showarrow=False,
    align="left",
    bgcolor="rgba(255,255,255,0.9)",
    bordercolor="gray",
    borderwidth=1
)

fig.show()

print("Score correlation analysis:")
print(f"Math vs Reading: {score_correlation.loc['math_score', 'reading_score']:.3f}")
print(f"Math vs Writing: {score_correlation.loc['math_score', 'writing_score']:.3f}")
print(f"Reading vs Writing: {score_correlation.loc['reading_score', 'writing_score']:.3f}")

Score correlation analysis:
Math vs Reading: 0.818
Math vs Writing: 0.803
Reading vs Writing: 0.955


In [14]:
# Interactive scatter plot matrix with regression lines
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=['Math vs Reading', 'Math vs Writing', 'Reading vs Writing'])

# Math vs Reading
fig.add_trace(go.Scatter(
    x=df['math_score'],
    y=df['reading_score'],
    mode='markers',
    name='Math vs Reading',
    marker=dict(size=6, opacity=0.6, color='blue'),
    hovertemplate="<b>Math Score:</b> %{x}<br><b>Reading Score:</b> %{y}<br><b>Gender:</b> %{customdata[0]}<br><b>Lunch:</b> %{customdata[1]}<extra></extra>",
    customdata=df[['gender', 'lunch']].values
), row=1, col=1)

# Add regression line
z = np.polyfit(df['math_score'], df['reading_score'], 1)
p = np.poly1d(z)
fig.add_trace(go.Scatter(
    x=sorted(df['math_score']),
    y=p(sorted(df['math_score'])),
    mode='lines',
    name='Trend Line',
    line=dict(color='red', dash='dash'),
    showlegend=False
), row=1, col=1)

# Math vs Writing
fig.add_trace(go.Scatter(
    x=df['math_score'],
    y=df['writing_score'],
    mode='markers',
    name='Math vs Writing',
    marker=dict(size=6, opacity=0.6, color='green'),
    hovertemplate="<b>Math Score:</b> %{x}<br><b>Writing Score:</b> %{y}<br><b>Gender:</b> %{customdata[0]}<br><b>Test Prep:</b> %{customdata[1]}<extra></extra>",
    customdata=df[['gender', 'test_prep']].values
), row=1, col=2)

z = np.polyfit(df['math_score'], df['writing_score'], 1)
p = np.poly1d(z)
fig.add_trace(go.Scatter(
    x=sorted(df['math_score']),
    y=p(sorted(df['math_score'])),
    mode='lines',
    line=dict(color='red', dash='dash'),
    showlegend=False
), row=1, col=2)

# Reading vs Writing
fig.add_trace(go.Scatter(
    x=df['reading_score'],
    y=df['writing_score'],
    mode='markers',
    name='Reading vs Writing',
    marker=dict(size=6, opacity=0.6, color='red'),
    hovertemplate="<b>Reading Score:</b> %{x}<br><b>Writing Score:</b> %{y}<br><b>Race/Ethnicity:</b> %{customdata[0]}<br><b>Education:</b> %{customdata[1]}<extra></extra>",
    customdata=df[['race_ethnicity', 'parental_education']].values
), row=1, col=3)

z = np.polyfit(df['reading_score'], df['writing_score'], 1)
p = np.poly1d(z)
fig.add_trace(go.Scatter(
    x=sorted(df['reading_score']),
    y=p(sorted(df['reading_score'])),
    mode='lines',
    line=dict(color='red', dash='dash'),
    showlegend=False
), row=1, col=3)

fig.update_layout(
    title_text="Interactive Score Correlation Scatter Plots with Regression Lines",
    height=500,
    showlegend=True
)

fig.update_xaxes(title="Score")
fig.update_yaxes(title="Score")

fig.show()

## 5. Interactive Comprehensive Analysis and Pattern Recognition

In [15]:
# Interactive multi-dimensional analysis: Interactive effects
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=[
                        'Gender + Test Preparation',
                        'Gender + Lunch Type', 
                        'Lunch Type + Test Preparation',
                        'Parental Education + Test Preparation'
                    ])

# Gender + Test preparation
gender_prep = df.groupby(['gender', 'test_prep'])['average_score'].mean().reset_index()
for test_prep in df['test_prep'].unique():
    data = gender_prep[gender_prep['test_prep'] == test_prep]
    fig.add_trace(go.Bar(
        x=data['gender'],
        y=data['average_score'],
        name=f'Test Prep: {test_prep}',
        hovertemplate="<b>Gender:</b> %{x}<br><b>Test Prep:</b> " + test_prep + "<br><b>Avg Score:</b> %{y:.2f}<extra></extra>"
    ), row=1, col=1)

# Gender + Lunch type
gender_lunch = df.groupby(['gender', 'lunch'])['average_score'].mean().reset_index()
for lunch in df['lunch'].unique():
    data = gender_lunch[gender_lunch['lunch'] == lunch]
    fig.add_trace(go.Bar(
        x=data['gender'],
        y=data['average_score'],
        name=f'Lunch: {lunch}',
        hovertemplate="<b>Gender:</b> %{x}<br><b>Lunch:</b> " + lunch + "<br><b>Avg Score:</b> %{y:.2f}<extra></extra>"
    ), row=1, col=2)

# Lunch type + Test preparation
lunch_prep = df.groupby(['lunch', 'test_prep'])['average_score'].mean().reset_index()
for test_prep in df['test_prep'].unique():
    data = lunch_prep[lunch_prep['test_prep'] == test_prep]
    fig.add_trace(go.Bar(
        x=data['lunch'],
        y=data['average_score'],
        name=f'Test Prep: {test_prep}',
        hovertemplate="<b>Lunch:</b> %{x}<br><b>Test Prep:</b> " + test_prep + "<br><b>Avg Score:</b> %{y:.2f}<extra></extra>"
    ), row=2, col=1)

# Parental education + Test preparation
edu_prep = df.groupby(['parental_education', 'test_prep'])['average_score'].mean().reset_index()
for test_prep in df['test_prep'].unique():
    data = edu_prep[edu_prep['test_prep'] == test_prep]
    fig.add_trace(go.Bar(
        x=data['parental_education'],
        y=data['average_score'],
        name=f'Test Prep: {test_prep}',
        hovertemplate="<b>Education:</b> %{x}<br><b>Test Prep:</b> " + test_prep + "<br><b>Avg Score:</b> %{y:.2f}<extra></extra>"
    ), row=2, col=2)

fig.update_layout(
    title_text="Interactive Multi-Dimensional Analysis: Factor Interactions",
    height=800,
    barmode='group',
    showlegend=False  # Too many legends would clutter
)

fig.update_xaxes(tickangle=45)
fig.update_yaxes(title="Average Score")

fig.show()

In [16]:
# Interactive score distribution and high-performer analysis
fig = make_subplots(rows=1, cols=2,
                    specs=[[{'type':'domain'}, {'type':'xy'}]],
                    subplot_titles=['Interactive Grade Distribution', 'High Performer Characteristics'])

# Interactive grade distribution pie chart
grade_counts = df['grade'].value_counts().sort_index()
colors = ['red', 'orange', 'yellow', 'lightgreen', 'green']

fig.add_trace(go.Pie(
    labels=grade_counts.index,
    values=grade_counts.values,
    name="Grade Distribution",
    marker_colors=colors,
    hovertemplate="<b>Grade %{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>",
    textinfo='label+percent'
), row=1, col=1)

# High-performer characteristics analysis
high_performers = df[df['average_score'] >= 80]
feature_analysis = []

features = ['gender', 'lunch', 'test_prep']
for feature in features:
    for category in df[feature].unique():
        total_in_category = len(df[df[feature] == category])
        high_in_category = len(high_performers[high_performers[feature] == category])
        percentage = (high_in_category / total_in_category) * 100 if total_in_category > 0 else 0
        feature_analysis.append({
            'Feature': feature,
            'Category': category, 
            'High_Performer_Ratio': percentage,
            'Count': high_in_category,
            'Total': total_in_category
        })

# Create grouped bar chart for high performer analysis
for feature in features:
    feature_data = [item for item in feature_analysis if item['Feature'] == feature]
    categories = [item['Category'] for item in feature_data]
    ratios = [item['High_Performer_Ratio'] for item in feature_data]
    
    fig.add_trace(go.Bar(
        x=categories,
        y=ratios,
        name=feature.replace('_', ' ').title(),
        hovertemplate="<b>" + feature.replace('_', ' ').title() + ":</b> %{x}<br><b>High Performer Ratio:</b> %{y:.1f}%<br><b>Count:</b> %{customdata[0]} / %{customdata[1]}<extra></extra>",
        customdata=[[item['Count'], item['Total']] for item in feature_data]
    ), row=1, col=2)

fig.update_layout(
    title_text="Interactive Grade Distribution and High Performer Analysis",
    height=500
)

fig.update_yaxes(title="High Performer Ratio (%)", row=1, col=2)
fig.update_xaxes(title="Category", tickangle=45, row=1, col=2)

fig.show()

high_performer_pct = len(high_performers)/len(df)*100
print(f"High performers (≥80 points) percentage: {high_performer_pct:.1f}%")
print(f"Total high performers: {len(high_performers)} students")

# Display top characteristics for high performers
print("\nCharacteristics most associated with high performance:")
for item in sorted(feature_analysis, key=lambda x: x['High_Performer_Ratio'], reverse=True)[:5]:
    print(f"  {item['Feature'].replace('_', ' ').title()} - {item['Category']}: {item['High_Performer_Ratio']:.1f}% high performers")

High performers (≥80 points) percentage: 19.8%
Total high performers: 198 students

Characteristics most associated with high performance:
  Test Prep - completed: 29.9% high performers
  Lunch - standard: 24.7% high performers
  Gender - female: 22.8% high performers
  Gender - male: 16.6% high performers
  Test Prep - none: 14.2% high performers


## 6. Interactive Key Insights Summary

In [17]:
# Interactive key statistics summary dashboard
print("=" * 60)
print("INTERACTIVE KEY FINDINGS STATISTICAL SUMMARY")
print("=" * 60)

# Calculate all key statistics
male_avg = df[df['gender'] == 'male']['average_score'].mean()
female_avg = df[df['gender'] == 'female']['average_score'].mean()
standard_avg = df[df['lunch'] == 'standard']['average_score'].mean()
reduced_avg = df[df['lunch'] == 'free/reduced']['average_score'].mean()
prep_completed = df[df['test_prep'] == 'completed']['average_score'].mean()
prep_none = df[df['test_prep'] == 'none']['average_score'].mean()
edu_impact = df.groupby('parental_education')['average_score'].mean()
masters_score = edu_impact["master's degree"]
high_school_score = edu_impact['some high school']
correlations = df[['math_score', 'reading_score', 'writing_score']].corr()
grade_distribution = df['grade'].value_counts().sort_index()

# Create interactive summary dashboard
summary_data = {
    'Metric': [
        'Male Average Score', 'Female Average Score', 'Gender Difference',
        'Standard Lunch Average', 'Free/Reduced Lunch Average', 'Economic Gap',
        'Test Prep Completed Average', 'No Test Prep Average', 'Test Prep Improvement',
        'Highest Education (Masters)', 'Lowest Education (Some HS)', 'Education Gap',
        'Math-Reading Correlation', 'Math-Writing Correlation', 'Reading-Writing Correlation'
    ],
    'Value': [
        f"{male_avg:.2f}", f"{female_avg:.2f}", f"{abs(female_avg - male_avg):.2f}",
        f"{standard_avg:.2f}", f"{reduced_avg:.2f}", f"{standard_avg - reduced_avg:.2f}",
        f"{prep_completed:.2f}", f"{prep_none:.2f}", f"{prep_completed - prep_none:.2f}",
        f"{masters_score:.2f}", f"{high_school_score:.2f}", f"{masters_score - high_school_score:.2f}",
        f"{correlations.loc['math_score', 'reading_score']:.3f}",
        f"{correlations.loc['math_score', 'writing_score']:.3f}",
        f"{correlations.loc['reading_score', 'writing_score']:.3f}"
    ],
    'Category': [
        'Gender', 'Gender', 'Gender',
        'Economic', 'Economic', 'Economic',
        'Test Prep', 'Test Prep', 'Test Prep',
        'Education', 'Education', 'Education',
        'Correlation', 'Correlation', 'Correlation'
    ]
}

# Create interactive table
fig = go.Figure(data=[go.Table(
    header=dict(values=['Metric', 'Value', 'Category'],
                fill_color='lightblue',
                align='left',
                font_size=14,
                height=40),
    cells=dict(values=[summary_data['Metric'], summary_data['Value'], summary_data['Category']],
               fill_color=[['white' if i % 2 == 0 else 'lightgray' for i in range(len(summary_data['Metric']))]]*3,
               align='left',
               font_size=12,
               height=30))
])

fig.update_layout(
    title="Interactive Statistical Summary Dashboard",
    height=600,
    margin=dict(l=0, r=0, t=50, b=0)
)

fig.show()

# Text summary for reference
print("\n1. Gender Differences:")
print(f"   - Male average score: {male_avg:.2f}")
print(f"   - Female average score: {female_avg:.2f}")
print(f"   - Difference: {abs(female_avg - male_avg):.2f} points")

print("\n2. Economic Status Impact:")
print(f"   - Standard lunch students average score: {standard_avg:.2f}")
print(f"   - Free/reduced lunch students average score: {reduced_avg:.2f}")
print(f"   - Difference: {standard_avg - reduced_avg:.2f} points")

print("\n3. Test Preparation Course Effect:")
print(f"   - Students who completed test prep average score: {prep_completed:.2f}")
print(f"   - Students who didn't take test prep average score: {prep_none:.2f}")
print(f"   - Improvement: {prep_completed - prep_none:.2f} points")

print("\n4. Parental Education Impact:")
print(f"   - Highest (master's degree): {masters_score:.2f}")
print(f"   - Lowest (some high school): {high_school_score:.2f}")
print(f"   - Difference: {masters_score - high_school_score:.2f} points")

print("\n5. Subject Correlations:")
print(f"   - Math vs Reading: {correlations.loc['math_score', 'reading_score']:.3f}")
print(f"   - Math vs Writing: {correlations.loc['math_score', 'writing_score']:.3f}")
print(f"   - Reading vs Writing: {correlations.loc['reading_score', 'writing_score']:.3f}")

print("\n6. Score Distribution:")
for grade, count in grade_distribution.items():
    percentage = (count / len(df)) * 100
    print(f"   - Grade {grade}: {count} students ({percentage:.1f}%)")

INTERACTIVE KEY FINDINGS STATISTICAL SUMMARY



1. Gender Differences:
   - Male average score: 65.84
   - Female average score: 69.57
   - Difference: 3.73 points

2. Economic Status Impact:
   - Standard lunch students average score: 70.84
   - Free/reduced lunch students average score: 62.20
   - Difference: 8.64 points

3. Test Preparation Course Effect:
   - Students who completed test prep average score: 72.67
   - Students who didn't take test prep average score: 65.04
   - Improvement: 7.63 points

4. Parental Education Impact:
   - Highest (master's degree): 73.60
   - Lowest (some high school): 65.11
   - Difference: 8.49 points

5. Subject Correlations:
   - Math vs Reading: 0.818
   - Math vs Writing: 0.803
   - Reading vs Writing: 0.955

6. Score Distribution:
   - Grade A: 52 students (5.2%)
   - Grade B: 146 students (14.6%)
   - Grade C: 261 students (26.1%)
   - Grade D: 256 students (25.6%)
   - Grade F: 285 students (28.5%)


## 7. Interactive Dashboard with Dash Components

### Complete Interactive Dashboard Implementation

In [18]:
# Complete Interactive Dash Dashboard (Improved hover and interactive multi-factor plot)
# Note: This cell creates a comprehensive dashboard that can be run in Jupyter
import webbrowser
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
# Define the layout
app.layout = dbc.Container([
    # Header
    dbc.Row([
        dbc.Col([
            html.H1("Student Performance Interactive Dashboard", 
                   className="text-center mb-4 text-primary"),
            html.Hr()
        ])
    ]),
    # Control Panel
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4("Interactive Filters", className="card-title"),
                    html.Label("Select Gender:"),
                    dcc.Dropdown(
                        id='gender-filter',
                        options=[{'label': 'All', 'value': 'all'}] + 
                               [{'label': gender, 'value': gender} for gender in df['gender'].unique()],
                        value='all',
                        multi=False # Only one selection at a time
                    ),
                    html.Br(),
                    html.Label("Select Lunch Type:"),
                    dcc.Dropdown(
                        id='lunch-filter',
                        options=[{'label': 'All', 'value': 'all'}] + 
                               [{'label': lunch, 'value': lunch} for lunch in df['lunch'].unique()],
                        value='all',
                        multi=False # Only one selection at a time
                    ),
                    html.Br(),
                    html.Label("Select Test Prep Status:"),
                    dcc.Dropdown(
                        id='testprep-filter',
                        options=[{'label': 'All', 'value': 'all'}] + 
                               [{'label': prep, 'value': prep} for prep in df['test_prep'].unique()],
                        value='all',
                        multi=False # Only one selection at a time
                    ),
                    html.Br(),
                    html.Label("Score Range:"),
                    dcc.RangeSlider(
                        id='score-range',
                        min=0,
                        max=100,
                        step=5,
                        marks={i: str(i) for i in range(0, 101, 20)},
                        value=[0, 100]
                    ),
                    html.Br(),
                    html.Label("Multi-Factor X Axis:"),
                    dcc.Dropdown(
                        id='multi-x',
                        options=[{'label': col, 'value': col} for col in ['average_score', 'math_score', 'reading_score', 'writing_score']],
                        value='average_score',
                        multi=False
                    ),
                    html.Br(),
                    html.Label("Multi-Factor Y Axis:"),
                    dcc.Dropdown(
                        id='multi-y',
                        options=[{'label': col, 'value': col} for col in ['math_score', 'reading_score', 'writing_score', 'average_score']],
                        value='math_score',
                        multi=False
                    ),
                    html.Br(),
                    html.Label("Multi-Factor Color:"),
                    dcc.Dropdown(
                        id='multi-color',
                        options=[{'label': col, 'value': col} for col in ['gender', 'lunch', 'test_prep']],
                        value='gender',
                        multi=False
                    )
                ])
            ])
        ], width=3),
        # Main Dashboard Area
        dbc.Col([
            # Summary Cards
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H4(id="total-students", className="card-title text-info"),
                            html.P("Total Students", className="card-text")
                        ])
                    ])
                ], width=3),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H4(id="avg-score", className="card-title text-success"),
                            html.P("Average Score", className="card-text")
                        ])
                    ])
                ], width=3),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H4(id="high-performers", className="card-title text-warning"),
                            html.P("High Performers (≥80)", className="card-text")
                        ])
                    ])
                ], width=3),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H4(id="improvement-rate", className="card-title text-danger"),
                            html.P("Test Prep Improvement", className="card-text")
                        ])
                    ])
                ], width=3)
            ], className="mb-4"),
            # Charts Row 1
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id="score-distribution-chart")
                ], width=6),
                dbc.Col([
                    dcc.Graph(id="gender-performance-chart")
                ], width=6)
            ], className="mb-4"),
            # Charts Row 2
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id="education-impact-chart")
                ], width=6),
                dbc.Col([
                    dcc.Graph(id="correlation-heatmap")
                ], width=6)
            ], className="mb-4"),
            # Charts Row 3
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id="multi-factor-analysis")
                ], width=12)
            ])
        ], width=9)
    ])
], fluid=True)
# Callback functions for interactivity
@app.callback(
    [
        Output('total-students', 'children'),
        Output('avg-score', 'children'),
        Output('high-performers', 'children'),
        Output('improvement-rate', 'children'),
        Output('score-distribution-chart', 'figure'),
        Output('gender-performance-chart', 'figure'),
        Output('education-impact-chart', 'figure'),
        Output('correlation-heatmap', 'figure'),
        Output('multi-factor-analysis', 'figure')
    ],
    [
        Input('gender-filter', 'value'),
        Input('lunch-filter', 'value'),
        Input('testprep-filter', 'value'),
        Input('score-range', 'value'),
        Input('multi-x', 'value'),
        Input('multi-y', 'value'),
        Input('multi-color', 'value')
    ]
)
def update_dashboard(gender_filter, lunch_filter, testprep_filter, score_range, multi_x, multi_y, multi_color):
    # Filter data based on selections (overwrite prior selection)
    filtered_df = df.copy()
    if gender_filter != 'all':
        filtered_df = filtered_df[filtered_df['gender'] == gender_filter]
    if lunch_filter != 'all':
        filtered_df = filtered_df[filtered_df['lunch'] == lunch_filter]
    if testprep_filter != 'all':
        filtered_df = filtered_df[filtered_df['test_prep'] == testprep_filter]
    filtered_df = filtered_df[
        (filtered_df['average_score'] >= score_range[0]) & 
        (filtered_df['average_score'] <= score_range[1])
    ]
    # Calculate summary statistics
    total_students = len(filtered_df)
    avg_score = f"{filtered_df['average_score'].mean():.1f}" if len(filtered_df) > 0 else "N/A"
    high_performers = len(filtered_df[filtered_df['average_score'] >= 80])
    # Test prep improvement calculation
    if len(filtered_df) > 0:
        prep_completed = filtered_df[filtered_df['test_prep'] == 'completed']['average_score'].mean()
        prep_none = filtered_df[filtered_df['test_prep'] == 'none']['average_score'].mean()
        improvement = f"+{prep_completed - prep_none:.1f}" if not (pd.isna(prep_completed) or pd.isna(prep_none)) else "N/A"
    else:
        improvement = "N/A"
    # 1. Score Distribution
    score_dist_fig = px.histogram(
        filtered_df, x='average_score', nbins=20,
        title="Score Distribution (Filtered)",
        labels={'average_score': 'Average Score', 'count': 'Number of Students'}
    )
    score_dist_fig.update_layout(height=400, hoverlabel_font_size=14)
    score_dist_fig.update_traces(hovertemplate="<b>Score:</b> %{x}<br><b>Count:</b> %{y}")
    # 2. Gender Performance (simplified hover)
    if len(filtered_df) > 0:
        gender_fig = px.box(
            filtered_df, x='gender', y='average_score', color='gender',
            color_discrete_map={'male': 'blue', 'female': 'pink'},
            title="Performance by Gender (Filtered)",
        )
        gender_fig.update_traces(hovertemplate="<b>Gender:</b> %{x}<br><b>Score:</b> %{y}")
    else:
        gender_fig = go.Figure()
        gender_fig.update_layout(title="No data available for current filters", height=400)
    gender_fig.update_layout(height=400, hoverlabel_font_size=14)
    # 3. Education Impact (simplified hover)
    if len(filtered_df) > 0:
        edu_fig = px.box(
            filtered_df, x='parental_education', y='average_score',
            title="Parental Education Impact (Filtered)",
        )
        edu_fig.update_xaxes(tickangle=45)
        edu_fig.update_traces(hovertemplate="<b>Education:</b> %{x}<br><b>Score:</b> %{y}")
    else:
        edu_fig = go.Figure()
        edu_fig.update_layout(title="No data available for current filters", height=400)
    edu_fig.update_layout(height=400, hoverlabel_font_size=14)
    # 4. Correlation Heatmap
    if len(filtered_df) > 0:
        corr_data = filtered_df[['math_score', 'reading_score', 'writing_score']].corr()
        corr_fig = px.imshow(
            corr_data, text_auto=True, aspect="auto",
            title="Subject Correlations (Filtered)"
        )
    else:
        corr_fig = px.imshow([[1]], title="No data available for current filters")
    corr_fig.update_layout(height=400, hoverlabel_font_size=14)
    # 5. Multi-factor Analysis (interactive plot)
    if len(filtered_df) > 0 and multi_x and multi_y and multi_color:
        multi_fig = px.scatter(
            filtered_df, x=multi_x, y=multi_y, color=multi_color,
            title=f"Multi-Factor Analysis: {multi_x} vs {multi_y} by {multi_color}",
            hover_data=[multi_x, multi_y, multi_color]
        )
        multi_fig.update_traces(marker=dict(size=10),
            hovertemplate=f'<b>{multi_x}:</b> %{{x}}<br><b>{multi_y}:</b> %{{y}}<br><b>{multi_color}:</b> %{{marker.color}}')
    else:
        multi_fig = go.Figure()
        multi_fig.update_layout(title="No data available for current filters", height=400)
    multi_fig.update_layout(height=400, hoverlabel_font_size=14)
    return (
        str(total_students),
        avg_score,
        str(high_performers),
        improvement,
        score_dist_fig,
        gender_fig,
        edu_fig,
        corr_fig,
        multi_fig
    )
if __name__ == '__main__':
    webbrowser.open('http://127.0.0.1:8050/')
    app.run(debug=True)
    print("App running at http://127.0.0.1:8050/")

App running at http://127.0.0.1:8050/


## 8. Enhanced Interactive Features and Recommendations

### Advanced Analytics Dashboard

In [ ]:
# Complete Interactive Dash Dashboard (Improved)
# Note: This cell creates a comprehensive dashboard that can be run in Jupyter
import webbrowser
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
# Define the layout
app.layout = dbc.Container([
    # Header
    dbc.Row([
        dbc.Col([
            html.H1("Student Performance Interactive Dashboard", 
                   className="text-center mb-4 text-primary"),
            html.Hr()
        ])
    ]),
    # Control Panel
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4("Interactive Filters", className="card-title"),
                    html.Label("Select Gender:"),
                    dcc.Dropdown(
                        id='gender-filter',
                        options=[{'label': 'All', 'value': 'all'}] + 
                               [{'label': gender, 'value': gender} for gender in df['gender'].unique()],
                        value='all',
                        multi=False # Only one selection at a time
                    ),
                    html.Br(),
                    html.Label("Select Lunch Type:"),
                    dcc.Dropdown(
                        id='lunch-filter',
                        options=[{'label': 'All', 'value': 'all'}] + 
                               [{'label': lunch, 'value': lunch} for lunch in df['lunch'].unique()],
                        value='all',
                        multi=False # Only one selection at a time
                    ),
                    html.Br(),
                    html.Label("Select Test Prep Status:"),
                    dcc.Dropdown(
                        id='testprep-filter',
                        options=[{'label': 'All', 'value': 'all'}] + 
                               [{'label': prep, 'value': prep} for prep in df['test_prep'].unique()],
                        value='all',
                        multi=False # Only one selection at a time
                    ),
                    html.Br(),
                    html.Label("Score Range:"),
                    dcc.RangeSlider(
                        id='score-range',
                        min=0,
                        max=100,
                        step=5,
                        marks={i: str(i) for i in range(0, 101, 20)},
                        value=[0, 100]
                    )
                ])
            ])
        ], width=3),
        # Main Dashboard Area
        dbc.Col([
            # Summary Cards
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H4(id="total-students", className="card-title text-info"),
                            html.P("Total Students", className="card-text")
                        ])
                    ])
                ], width=3),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H4(id="avg-score", className="card-title text-success"),
                            html.P("Average Score", className="card-text")
                        ])
                    ])
                ], width=3),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H4(id="high-performers", className="card-title text-warning"),
                            html.P("High Performers (≥80)", className="card-text")
                        ])
                    ])
                ], width=3),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H4(id="improvement-rate", className="card-title text-danger"),
                            html.P("Test Prep Improvement", className="card-text")
                        ])
                    ])
                ], width=3)
            ], className="mb-4"),
            # Charts Row 1
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id="score-distribution-chart")
                ], width=6),
                dbc.Col([
                    dcc.Graph(id="gender-performance-chart")
                ], width=6)
            ], className="mb-4"),
            # Charts Row 2
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id="education-impact-chart")
                ], width=6),
                dbc.Col([
                    dcc.Graph(id="correlation-heatmap")
                ], width=6)
            ], className="mb-4"),
            # Charts Row 3
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id="multi-factor-analysis")
                ], width=12)
            ])
        ], width=9)
    ])
], fluid=True)
# Callback functions for interactivity
@app.callback(
    [
        Output('total-students', 'children'),
        Output('avg-score', 'children'),
        Output('high-performers', 'children'),
        Output('improvement-rate', 'children'),
        Output('score-distribution-chart', 'figure'),
        Output('gender-performance-chart', 'figure'),
        Output('education-impact-chart', 'figure'),
        Output('correlation-heatmap', 'figure'),
        Output('multi-factor-analysis', 'figure')
    ],
    [
        Input('gender-filter', 'value'),
        Input('lunch-filter', 'value'),
        Input('testprep-filter', 'value'),
        Input('score-range', 'value')
    ]
)
def update_dashboard(gender_filter, lunch_filter, testprep_filter, score_range):
    # Filter data based on selections (overwrite prior selection)
    filtered_df = df.copy()
    if gender_filter != 'all':
        filtered_df = filtered_df[filtered_df['gender'] == gender_filter]
    if lunch_filter != 'all':
        filtered_df = filtered_df[filtered_df['lunch'] == lunch_filter]
    if testprep_filter != 'all':
        filtered_df = filtered_df[filtered_df['test_prep'] == testprep_filter]
    filtered_df = filtered_df[
        (filtered_df['average_score'] >= score_range[0]) & 
        (filtered_df['average_score'] <= score_range[1])
    ]
    # Calculate summary statistics
    total_students = len(filtered_df)
    avg_score = f"{filtered_df['average_score'].mean():.1f}" if len(filtered_df) > 0 else "N/A"
    high_performers = len(filtered_df[filtered_df['average_score'] >= 80])
    # Test prep improvement calculation
    if len(filtered_df) > 0:
        prep_completed = filtered_df[filtered_df['test_prep'] == 'completed']['average_score'].mean()
        prep_none = filtered_df[filtered_df['test_prep'] == 'none']['average_score'].mean()
        improvement = f"+{prep_completed - prep_none:.1f}" if not (pd.isna(prep_completed) or pd.isna(prep_none)) else "N/A"
    else:
        improvement = "N/A"
    # 1. Score Distribution
    score_dist_fig = px.histogram(
        filtered_df, x='average_score', nbins=20,
        title="Score Distribution (Filtered)",
        labels={'average_score': 'Average Score', 'count': 'Number of Students'}
    )
    score_dist_fig.update_layout(height=400, hoverlabel_font_size=14)
    score_dist_fig.update_traces(hovertemplate="<b>Score:</b> %{x}<br><b>Count:</b> %{y}")
    # 2. Gender Performance (differentiate by color, improve hover, no repetition)
    if len(filtered_df) > 0:
        gender_fig = px.box(
            filtered_df, x='gender', y='average_score', color='gender',
            color_discrete_map={'male': 'blue', 'female': 'pink'},
            title="Performance by Gender (Filtered)",
            hover_data=filtered_df.columns
        )
        gender_fig.update_traces(hovertemplate="<b>Gender:</b> %{x}<br><b>Score:</b> %{y}")
    else:
        gender_fig = go.Figure()
        gender_fig.update_layout(title="No data available for current filters", height=400)
    gender_fig.update_layout(height=400, hoverlabel_font_size=14)
    # 3. Education Impact
    if len(filtered_df) > 0:
        edu_fig = px.box(
            filtered_df, x='parental_education', y='average_score',
            title="Parental Education Impact (Filtered)",
            hover_data=filtered_df.columns
        )
        edu_fig.update_xaxes(tickangle=45)
        edu_fig.update_traces(hovertemplate="<b>Education:</b> %{x}<br><b>Score:</b> %{y}")
    else:
        edu_fig = go.Figure()
        edu_fig.update_layout(title="No data available for current filters", height=400)
    edu_fig.update_layout(height=400, hoverlabel_font_size=14)
    # 4. Correlation Heatmap
    if len(filtered_df) > 0:
        corr_data = filtered_df[['math_score', 'reading_score', 'writing_score']].corr()
        corr_fig = px.imshow(
            corr_data, text_auto=True, aspect="auto",
            title="Subject Correlations (Filtered)"
        )
    else:
        corr_fig = px.imshow([[1]], title="No data available for current filters")
    corr_fig.update_layout(height=400, hoverlabel_font_size=14)
    # 5. Multi-factor Analysis (fix: use scatter for clarity, color by gender, improve hover)
    if len(filtered_df) > 0:
        multi_fig = px.scatter(
            filtered_df, x='average_score', y='math_score', color='gender',
            color_discrete_map={'male': 'blue', 'female': 'pink'},
            title="Multi-Factor Analysis: Average vs Math Score by Gender",
            hover_data=filtered_df.columns
        )
        multi_fig.update_traces(marker=dict(size=10),
            hovertemplate="<b>Score:</b> %{x}<br><b>Math:</b> %{y}<br><b>Gender:</b> %{marker.color}")
    else:
        multi_fig = go.Figure()
        multi_fig.update_layout(title="No data available for current filters", height=400)
    multi_fig.update_layout(height=400, hoverlabel_font_size=14)
    return (
        str(total_students),
        avg_score,
        str(high_performers),
        improvement,
        score_dist_fig,
        gender_fig,
        edu_fig,
        corr_fig,
        multi_fig
    )
if __name__ == '__main__':
    webbrowser.open('http://127.0.0.1:8050/')
    app.run(debug=True)
    print("App running at http://127.0.0.1:8050/")

App running at http://127.0.0.1:8050/


In [ ]:
from scipy.stats import ttest_ind
from scipy.stats import f_oneway
from sklearn.ensemble import RandomForestRegressor

# Statistical and analytical methods applied to the student performance data

# 1. Descriptive statistics
desc_stats = df.describe(include='all')
print("Descriptive Statistics:\n", desc_stats)

# 2. Grouped means and differences
gender_means = df.groupby('gender')['average_score'].mean()
print("\nAverage score by gender:\n", gender_means)

lunch_means = df.groupby('lunch')['average_score'].mean()
print("\nAverage score by lunch type:\n", lunch_means)

prep_means = df.groupby('test_prep')['average_score'].mean()
print("\nAverage score by test preparation:\n", prep_means)

# 3. T-tests for group differences

# Gender difference
t_gender, p_gender = ttest_ind(df[df['gender'] == 'female']['average_score'],
                               df[df['gender'] == 'male']['average_score'])
print(f"\nT-test Gender Difference: t={t_gender:.3f}, p={p_gender:.3f}")

# Lunch type difference
t_lunch, p_lunch = ttest_ind(df[df['lunch'] == 'standard']['average_score'],
                             df[df['lunch'] == 'free/reduced']['average_score'])
print(f"T-test Lunch Type Difference: t={t_lunch:.3f}, p={p_lunch:.3f}")

# Test prep difference
t_prep, p_prep = ttest_ind(df[df['test_prep'] == 'completed']['average_score'],
                           df[df['test_prep'] == 'none']['average_score'])
print(f"T-test Test Prep Difference: t={t_prep:.3f}, p={p_prep:.3f}")

# 4. Correlation analysis
score_corr = df[['math_score', 'reading_score', 'writing_score', 'average_score']].corr()
print("\nScore Correlation Matrix:\n", score_corr)

# 5. ANOVA for parental education impact

edu_groups = [df[df['parental_education'] == level]['average_score'] for level in education_order]
f_edu, p_edu = f_oneway(*edu_groups)
print(f"\nANOVA Parental Education Impact: F={f_edu:.3f}, p={p_edu:.3f}")

# 6. Feature importance using RandomForest (for prediction)

# Encode categorical variables
df_rf = df.copy()
df_rf['gender'] = df_rf['gender'].map({'female': 0, 'male': 1})
df_rf['lunch'] = df_rf['lunch'].map({'standard': 0, 'free/reduced': 1})
df_rf['test_prep'] = df_rf['test_prep'].map({'none': 0, 'completed': 1})
df_rf['parental_education'] = df_rf['parental_education'].cat.codes

features_rf = ['gender', 'race_ethnicity', 'parental_education', 'lunch', 'test_prep', 'math_score', 'reading_score', 'writing_score']
X_rf = pd.get_dummies(df_rf[features_rf], drop_first=True)
y_rf = df_rf['average_score']

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_rf, y_rf)
importances = pd.Series(rf.feature_importances_, index=X_rf.columns).sort_values(ascending=False)
print("\nFeature Importances (RandomForest):\n", importances)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Interactive Scenario Analysis Tool

# Create scenario comparison visualization
def create_scenario_comparison():
    scenarios = {
        'Baseline (Current)': df,
        'If All Had Test Prep': df.copy(),
        'If All Had Standard Lunch': df.copy(),
        'Optimal Scenario': df.copy()
    }
    
    # Modify scenarios
    scenarios['If All Had Test Prep']['test_prep'] = 'completed'
    scenarios['If All Had Standard Lunch']['lunch'] = 'standard'
    scenarios['Optimal Scenario']['test_prep'] = 'completed'
    scenarios['Optimal Scenario']['lunch'] = 'standard'
    
    # Calculate scenario impacts
    scenario_results = []
    for scenario_name, scenario_df in scenarios.items():
        # Recalculate scores based on observed improvements
        if 'Test Prep' in scenario_name or scenario_name == 'Optimal Scenario':
            improvement = prep_completed - prep_none  # From previous calculations
            mask = (scenario_df['test_prep'] == 'completed') & (df['test_prep'] == 'none')
            scenario_df.loc[mask, 'average_score'] += improvement
        
        if 'Standard Lunch' in scenario_name or scenario_name == 'Optimal Scenario':
            improvement = standard_avg - reduced_avg
            mask = (scenario_df['lunch'] == 'standard') & (df['lunch'] == 'free/reduced')
            scenario_df.loc[mask, 'average_score'] += improvement
        
        avg_score = scenario_df['average_score'].mean()
        high_performers = len(scenario_df[scenario_df['average_score'] >= 80])
        
        scenario_results.append({
            'Scenario': scenario_name,
            'Average Score': avg_score,
            'High Performers': high_performers,
            'High Performer %': (high_performers / len(scenario_df)) * 100
        })
    
    scenario_df_results = pd.DataFrame(scenario_results)
    
    # Create comparison visualization
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=['Average Score by Scenario', 'High Performers by Scenario'])
    
    # Average scores
    fig.add_trace(go.Bar(
        x=scenario_df_results['Scenario'],
        y=scenario_df_results['Average Score'],
        name='Average Score',
        marker_color='lightblue',
        hovertemplate="<b>Scenario:</b> %{x}<br><b>Average Score:</b> %{y:.2f}<extra></extra>"
    ), row=1, col=1)
    
    # High performers percentage
    fig.add_trace(go.Bar(
        x=scenario_df_results['Scenario'],
        y=scenario_df_results['High Performer %'],
        name='High Performer %',
        marker_color='lightgreen',
        hovertemplate="<b>Scenario:</b> %{x}<br><b>High Performers:</b> %{y:.1f}%<extra></extra>"
    ), row=1, col=2)
    
    fig.update_layout(
        title_text="Interactive Scenario Analysis: Policy Impact Simulation",
        height=500,
        showlegend=False
    )
    
    fig.update_xaxes(tickangle=45)
    fig.update_yaxes(title="Average Score", row=1, col=1)
    fig.update_yaxes(title="High Performer Percentage", row=1, col=2)
    
    return fig, scenario_df_results

scenario_fig, scenario_results = create_scenario_comparison()
scenario_fig.show()

# Display scenario analysis results
print("\nScenario Analysis Results:")
print("=" * 50)
for _, row in scenario_results.iterrows():
    print(f"{row['Scenario']}:")
    print(f"  Average Score: {row['Average Score']:.2f}")
    print(f"  High Performers: {row['High Performers']} ({row['High Performer %']:.1f}%)")
    if row['Scenario'] != 'Baseline (Current)':
        baseline_score = scenario_results[scenario_results['Scenario'] == 'Baseline (Current)']['Average Score'].iloc[0]
        baseline_high = scenario_results[scenario_results['Scenario'] == 'Baseline (Current)']['High Performer %'].iloc[0]
        print(f"  Improvement: +{row['Average Score'] - baseline_score:.2f} points")
        print(f"  High Performer Increase: +{row['High Performer %'] - baseline_high:.1f}%")
    print()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## 9. Interactive Recommendations and Conclusions

### Data-Driven Policy Recommendations with Interactive Evidence

In [ ]:
# Create interactive recommendations dashboard

# Key findings with interactive evidence
key_findings = {
    'Socioeconomic Impact': {
        'finding': f"Students with standard lunch outperform free/reduced lunch students by {standard_avg - reduced_avg:.1f} points",
        'evidence': f"Standard lunch average: {standard_avg:.2f}, Free/reduced average: {reduced_avg:.2f}",
        'recommendation': "Implement comprehensive support programs for economically disadvantaged students",
        'priority': 'High'
    },
    'Test Preparation Effectiveness': {
        'finding': f"Test preparation courses improve scores by {prep_completed - prep_none:.1f} points on average",
        'evidence': f"With prep: {prep_completed:.2f}, Without prep: {prep_none:.2f}",
        'recommendation': "Expand access to test preparation programs, especially for disadvantaged groups",
        'priority': 'High'
    },
    'Parental Education Correlation': {
        'finding': f"Strong positive correlation ({correlation:.3f}) between parental education and student performance",
        'evidence': f"Range from {high_school_score:.2f} (some high school) to {masters_score:.2f} (master's degree)",
        'recommendation': "Develop parent engagement and education programs",
        'priority': 'Medium'
    },
    'Subject Integration': {
        'finding': f"Very high correlation between reading and writing ({correlations.loc['reading_score', 'writing_score']:.3f})",
        'evidence': "Strong inter-subject correlations suggest integrated skill development",
        'recommendation': "Implement integrated curriculum focusing on cross-subject skill building",
        'priority': 'Medium'
    },
    'Gender Considerations': {
        'finding': f"Females outperform males by {female_avg - male_avg:.1f} points overall",
        'evidence': "Differences vary by subject: females excel in language arts, males in mathematics",
        'recommendation': "Develop gender-specific support strategies for different subjects",
        'priority': 'Low'
    }
}

# Create interactive recommendations table
recommendations_data = []
for category, details in key_findings.items():
    recommendations_data.append({
        'Category': category,
        'Key Finding': details['finding'],
        'Evidence': details['evidence'],
        'Recommendation': details['recommendation'],
        'Priority': details['priority']
    })

recommendations_df = pd.DataFrame(recommendations_data)

# Create interactive table
fig = go.Figure(data=[go.Table(
    header=dict(values=list(recommendations_df.columns),
                fill_color='darkblue',
                font_color='white',
                align='left',
                font_size=12,
                height=40),
    cells=dict(values=[recommendations_df[col] for col in recommendations_df.columns],
               fill_color=[['lightblue' if details['priority'] == 'High' 
                          else 'lightyellow' if details['priority'] == 'Medium' 
                          else 'lightgray' for details in key_findings.values()]]*5,
               align='left',
               font_size=11,
               height=60))
])

fig.update_layout(
    title="Interactive Policy Recommendations Dashboard",
    height=500,
    margin=dict(l=0, r=0, t=50, b=0)
)

fig.show()

# Create priority-based action plan visualization
priority_counts = recommendations_df['Priority'].value_counts()
fig_priority = px.pie(values=priority_counts.values, names=priority_counts.index,
                     title="Recommendation Priorities Distribution",
                     color_discrete_map={'High': 'red', 'Medium': 'orange', 'Low': 'green'})
fig_priority.show()

print("\n" + "=" * 80)
print("INTERACTIVE POLICY RECOMMENDATIONS SUMMARY")
print("=" * 80)
print("\nHIGH PRIORITY ACTIONS:")
for category, details in key_findings.items():
    if details['priority'] == 'High':
        print(f"\n• {category}:")
        print(f"  Finding: {details['finding']}")
        print(f"  Action: {details['recommendation']}")

print("\nMEDIUM PRIORITY ACTIONS:")
for category, details in key_findings.items():
    if details['priority'] == 'Medium':
        print(f"\n• {category}:")
        print(f"  Finding: {details['finding']}")
        print(f"  Action: {details['recommendation']}")

print("\nLOW PRIORITY ACTIONS:")
for category, details in key_findings.items():
    if details['priority'] == 'Low':
        print(f"\n• {category}:")
        print(f"  Finding: {details['finding']}")
        print(f"  Action: {details['recommendation']}")


INTERACTIVE POLICY RECOMMENDATIONS SUMMARY

HIGH PRIORITY ACTIONS:

• Socioeconomic Impact:
  Finding: Students with standard lunch outperform free/reduced lunch students by 8.6 points
  Action: Implement comprehensive support programs for economically disadvantaged students

• Test Preparation Effectiveness:
  Finding: Test preparation courses improve scores by 7.6 points on average
  Action: Expand access to test preparation programs, especially for disadvantaged groups

MEDIUM PRIORITY ACTIONS:

• Parental Education Correlation:
  Finding: Strong positive correlation (0.939) between parental education and student performance
  Action: Develop parent engagement and education programs

• Subject Integration:
  Finding: Very high correlation between reading and writing (0.955)
  Action: Implement integrated curriculum focusing on cross-subject skill building

LOW PRIORITY ACTIONS:

• Gender Considerations:
  Finding: Females outperform males by 3.7 points overall
  Action: Develop gen

## 10. Conclusion and Next Steps

### Interactive Student Performance Analysis - Key Takeaways

**This interactive analysis using Plotly and Dash has revealed significant insights into student performance patterns:**

### **Major Findings:**

1. **Economic Status is the Strongest Predictor** - Students from higher socioeconomic backgrounds (standard lunch) outperform their peers by 8.6 points on average

2. **Test Preparation Shows Clear Benefits** - Students who completed test prep courses improved by 7.6 points across all subjects

3. **Parental Education Matters** - Strong positive correlation (0.xxx) between parental education level and student achievement

4. **Subject Skills are Interconnected** - Very high correlation (0.955) between reading and writing suggests integrated learning approaches

5. **Gender Differences are Subject-Specific** - Females excel in language arts while males show slight advantages in mathematics

### **Interactive Features Implemented:**

- **Real-time Filtering**: Dynamic charts that update based on demographic selections
- **Hover Information**: Detailed data points with contextual information
- **Statistical Overlays**: T-test results, correlation coefficients, and confidence intervals
- **Scenario Analysis**: Predictive modeling for policy impact assessment
- **Responsive Dashboard**: Professional layout with Bootstrap styling

### **Technical Implementation:**

- **Plotly Express & Graph Objects**: For interactive visualizations
- **Dash Components**: For dashboard creation and callbacks
- **Statistical Analysis**: Integration of scipy.stats for hypothesis testing
- **Machine Learning**: RandomForest for feature importance and prediction
- **Data Transformation**: Comprehensive preprocessing for optimal visualization

### **Recommended Next Steps:**

1. **Implement High-Priority Interventions** focusing on socioeconomic support
2. **Expand Test Preparation Access** particularly for underserved populations
3. **Develop Parent Engagement Programs** to leverage education correlation
4. **Create Integrated Curriculum** capitalizing on subject interconnections
5. **Monitor and Evaluate** using this interactive dashboard framework

**This interactive analysis framework can be adapted for ongoing monitoring and policy evaluation, providing stakeholders with real-time insights into student performance trends and intervention effectiveness.**